# Member's Machine Actionable Data Management Plans

Data management plans (DMPs) are documents accompanying research proposals and project outputs. DMPs are created as free-form text and describe the data and tools employed in scientific investigations. They are often seen as an administrative exercise and not as an integral part of research practice. Machine Actionable DMPs (maDMPs) takes this concept further by 

This notebook displays in a human-friendly all the DMPs related to Member. By the end of this notebook, you will be able to succinctly display all the DMPs related to a DataCite Member.


In [118]:
import json
import pandas as pd
import numpy as np
from dfply import *
import altair.vega.v5 as alt


In [119]:
# Prepare the GraphQL client
import requests
from IPython.display import display, Markdown
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

_transport = RequestsHTTPTransport(
    url='https://api.datacite.org/graphql',
    use_json=True,
)

client = Client(
    transport=_transport,
    fetch_schema_from_transport=True,
)

In [120]:
# Generate the GraphQL query to retrieve up to 100 outputs of University of Oxford, with at least 100 views each.
query_params = {
    "memberId" : "cern"
}

query = gql("""query getOutputs($memberId: String)
{
  dataManagementPlans(memberId: $memberId) {
    totalCount
    nodes {
      id
      title: titles(first: 1) {
        title
      }
      datasets: references(resourceTypeId: "dataset") {
        totalCount
        nodes {
          id: doi
          name: titles(first: 1) {
            title
          }
        }
      }
      organisations: contributors(contributorType: "HostingInstitution") {
        id
        name
        affiliation {
          id
        }
      }
      funders: fundingReferences {
        id: funderIdentifier
        funderIdentifierType
        name: funderName
      }
      people: creators {
        id
        name
        affiliation {
          id
        }
      }
    }
  }
}
""")

In [121]:
def get_series_size(series_element):
    return len(series_element)

In [122]:
def get_dataset_nodes(series_element):
    return series_element['nodes']

In [123]:
def get_title(series_element):
    return series_element[0]['title']

In [124]:
def transform_dmps(dataframe):
    """Modifies each item to include attributes needed for the node visulisation

    Parameters:
    dataframe (dataframe): A dataframe with all the itemss
    parent (int): The id of the parent node

    Returns:
    dataframe:Returning vthe same dataframe with new attributes

   """
    if (dataframe) is None:
        return pd.DataFrame() 
    else: 
        dataframe = (dataframe >>
        mutate(
            datasets = X.datasets.apply(get_dataset_nodes)
        ))

        return (dataframe >>
        mutate(
            id = X.id,
            title = X.title.apply(get_title),
            datasets = X.datasets.apply(get_series_size),
            organisations = X.organisations.apply(get_series_size),
            funders = X.funders.apply(get_series_size),
            people = X.people.apply(get_series_size)
        ))
  

In [125]:
def get_data():
    return client.execute(query, variable_values=json.dumps(query_params))["dataManagementPlans"]

In [116]:
data = get_data()["nodes"]

table = pd.DataFrame(data,columns=data[0].keys())

In [117]:
transform_dmps(table).style

,id,title,datasets,organisations,funders,people
0,https://doi.org/10.5281/zenodo.2655759,Periódicos técnicos,0,0,0,1
1,https://doi.org/10.5281/zenodo.2655758,Periódicos técnicos,0,0,0,1
2,https://doi.org/10.5281/zenodo.2741388,Fractional-order functions for solving fractional-order variational problems with boundary conditions,0,0,0,2
3,https://doi.org/10.5281/zenodo.2741387,Fractional-order functions for solving fractional-order variational problems with boundary conditions,0,0,0,2
4,https://doi.org/10.5281/zenodo.3245354,EURHISFIRM D1.2: Data Management Plan (first version),0,0,1,3
5,https://doi.org/10.5281/zenodo.3245353,EURHISFIRM D1.2: Data Management Plan (first version),0,0,1,3
6,https://doi.org/10.5281/zenodo.3246339,EURHISFIRM D1.7: Second Data Management Plan,0,0,1,5
7,https://doi.org/10.5281/zenodo.3246338,EURHISFIRM D1.7: Second Data Management Plan,0,0,1,5
8,https://doi.org/10.5281/zenodo.3247755,Example ezDMP output,0,0,0,1
9,https://doi.org/10.5281/zenodo.3247756,Example ezDMP output,0,0,0,1
